# Convergence Benchmarks

In [3]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Fit SPAM model

### Generate True Model, Full POVM and Inital

In [11]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def spam_pipeline(n, 
                  c1, 
                  c2,
                  noise,
                  use_corr = False):
    spam_target = generate_spam_benchmark(n=n, c1=c1, c2=c2)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    
    if use_corr:
        povm = CorruptionMatrix(d, c=None)
    else:
        povm = POVM(d, c=None)
        
    spam_model = SPAM(init = InitialState(d, c=None),
                      povm = povm,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
        
    spam_model.pretrain(num_iter = 300,
                     verbose = False,
                    )

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )
    
    return spam_target, spam_model


def povm_fidelity(povm_a, povm_b):
    d = povm_a.shape[0]
    ab = tf.matmul(povm_a, povm_b)
    ab_sqrt = tf.linalg.sqrtm(ab)
    fidelity = tf.math.reduce_sum(tf.linalg.trace(ab_sqrt))/d
    return fidelity

## Three Qubit

In [14]:
n = 3
d = 2**n
c1 = 0.8
c2 = 0.8

num_reps = 2
noise_list = np.logspace(-1, -3, 3)
noise_list = np.append(noise_list, 0)

init_fid_list = np.zeros((len(noise_list), num_reps))
povm_fid_list = np.zeros((len(noise_list), num_reps))

for i, noise in tqdm(enumerate(noise_list)):
    
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                noise = noise,
                                                use_corr=False,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))

0it [00:00, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.005109909216313884


C:\Program Files\Anaconda3\envs\env_qiskit\lib\site-packages\tensorflow\python\framework\ops.py:1062: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self._numpy())


...

  0%|          | 0/2000 [00:00<?, ?it/s]

1.8057316467220857e-07


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

1.6000615715865896e-05


In [21]:
n = 3
d = 2**n
c1 = 0.8
c2 = 0.8

num_reps = 2
noise_list = np.logspace(-1, -3, 3)
noise_list = np.append(noise_list, 0)

init_fid_list = np.zeros((len(noise_list), num_reps))
povm_fid_list = np.zeros((len(noise_list), num_reps))

for i, noise in tqdm(enumerate(noise_list)):
    
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                noise = noise,
                                                use_corr=True,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_corr.data"))

0it [00:00, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0067154864182469584


  0%|          | 0/300 [00:00<?, ?it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0002224422689971035


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.00022359843262244832


## Four Qubit

In [22]:
n = 4
d = 2**n
c1 = 0.8
c2 = 0.8

num_reps = 2
noise_list = np.logspace(-1, -3, 3)
noise_list = np.append(noise_list, 0)

init_fid_list = np.zeros((len(noise_list), num_reps))
povm_fid_list = np.zeros((len(noise_list), num_reps))

for i, noise in tqdm(enumerate(noise_list)):
    
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                noise = noise,
                                                use_corr=False,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_4qubit_full.data"))

0it [00:00, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0069867701793989755


  0%|          | 0/300 [00:00<?, ?it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

8.333051316350932e-06


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

4.933690428900998e-07


In [23]:
n = 4
d = 2**n
c1 = 0.8
c2 = 0.8

num_reps = 2
noise_list = np.logspace(-1, -3, 3)
noise_list = np.append(noise_list, 0)

init_fid_list = np.zeros((len(noise_list), num_reps))
povm_fid_list = np.zeros((len(noise_list), num_reps))

for i, noise in tqdm(enumerate(noise_list)):
    
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                noise = noise,
                                                use_corr=True,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_4qubit_corr.data"))

0it [00:00, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 